# Introduction
**This Jupiter Notebook created for the purposes of setting up and running Ai models with [Verbum OCR](https://github.com/ragelmalti/verbum_ocr).**

VerbumOCR is capable of running various Ai models to perform OCR. It utilises the OpenAI Python library, to make API requests to any Ai model that utilises the OpenAI API specification.

This notebook will be split into two parts:
1. Instructions for setting up VerbumOCR with **popular properitary models**, including ChatGPT and Google Gemini
2. Instructions and code for hosting **locally hosted models from HuggingFace** using either **vLLM or Ollama.**
  - **Note:** It's suggested if you run the code in this Jupiter Notebook, that you utilise Google Colab's Nvidia A100 40GB GPU for the best results. Otherwise, a high performance GPU on a local machine will substitute.

**IMPORTANT:** A `config.env` file needs to be created in the base directory where the `verbum_ocr.py` script will be run.

You'll need to add/change the `LLM_BASE_URL` and `LLM_API_KEY` env variables depending on what model you're using.

# #1 Setting up properitary models
## Google Gemini
*Instructions adapted from: https://ai.google.dev/gemini-api/docs/openai*

*   Generate an API key with [Google Ai Studio](https://aistudio.google.com/apikey).
*   Set the `LLM_API_KEY` env variable in `config.env` to include the API key generated prior.
*   Set the `LLM_BASE_URL` env variable to: "https://generativelanguage.googleapis.com/v1beta/openai/"
*   When running the `verbum_ocr.py` script, set the `--model_name` flag to include a Gemini model, e.g. `gemini-pro-vision` or `gemini-2.5-flash`


## ChatGPT
**NOTE:** Make sure you add credits via the OpenAPI Platform portal: https://platform.openai.com/settings/organization/billing/overview

*Instructions adapted from: https://platform.openai.com/docs/api-reference/responses/create*
*   Generate an API key with [OpenAI Platform](https://platform.openai.com/api-keys)
*   Set the `LLM_API_KEY` env variable in `config.env` to include the API key generated prior.
*   Set the `LLM_BASE_URL` env variable to: "https://api.openai.com/v1/responses"
*   When running the `verbum_ocr.py` script, set the `--model_name` flag to include a ChatGPT model, e.g. `gpt-5-nano`


---

# #2 Setting up local hosted models from HuggingFace

## Python PIP Dependencies for vLLM and Verbum OCR

In [ ]:
# vLLM Dependencies
!pip install openai>=1.52.2
!pip install vllm>=0.6.3
!pip install triton>=3.1.0
!pip install nest_asyncio # only needed in colab
# Verbum OCR Dependencies
!pip install jiwer
!pip install pymupdf
!pip install requests
!pip install python-dotenv
!pip install google-genai
# Ngrok dependecy
!pip install pyngrok
!pip check

In [ ]:
# This should not be necessary outside of colab.
import nest_asyncio
nest_asyncio.apply()

## Installing Ollama
Adapted from https://colab.research.google.com/github/5aharsh/collama/blob/main/Ollama_Setup.ipynb

In [ ]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

## Running via Ngrok

Ngrok will be used to as a way to "port foward" the Ollama or vLLM instance to the internet.

The pyngrok library is used: https://pyngrok.readthedocs.io/en/latest/integrations.html#colab-ssh-example


Set the `LLM_BASE_URL` var in `config.env` to the ngrok tunnel URL, with the endpoint, /v1 at the end.

Set `LLM_API_KEY=123`

E.g. `LLM_BASE_URL=https://49c194b286d2.ngrok-free.app/v1`

## Running on local machine

If the ai model is running on your local machine, set the `LLM_BASE_URL` var in `config.env` to `LLM_BASE_URL=http://127.0.0.1/8000/v1`

Set `LLM_API_KEY=123`

## Execute vLLM
Link to the vLLM instructions for Google Colab: https://cloud.google.com/dataflow/docs/notebooks/run_inference_vllm

### Vision models tested with vLLM
Models are stored in `/root/.cache/huggingface/hub`

* ✅ Qwen/Qwen2.5-VL-3B-Instruct
* ✅ nanonets/Nanonets-OCR-s
* ✅ llava-hf/llava-v1.6-mistral-7b-hf
* ✅ ibm-granite/granite-vision-3.2-2b
* ✅ allenai/olmOCR-7B-0225-preview
* ✅ ChatDOC/OCRFlux-3B
* ✅ reducto/RolmOCR
* ? rednote-hilab/dots.ocr
* ❌ deepseek-ai/deepseek-vl2-tiny
* ❌ google/gemma-3-4b-it




In [ ]:
from pyngrok import ngrok
from pyngrok import conf
from google.colab import userdata
import os

# = NGROK SETUP =
# Auth token copied from https://dashboard.ngrok.com/get-started/your-authtoken
os.environ["NGROK"] = userdata.get("NGROK")
conf.get_default().auth_token = os.environ["NGROK"]
port = 8000
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# = START vLLM =

!python -m vllm.entrypoints.openai.api_server --trust-remote-code --model reducto/RolmOCR

## Execute Ollama
Check https://ollama.com/search?c=vision for a list of vision models

In [ ]:
from pyngrok import ngrok
from pyngrok import conf
from google.colab import userdata

# = NGROK SETUP =
# Auth token copied from https://dashboard.ngrok.com/get-started/your-authtoken
os.environ["NGROK"] = userdata.get("NGROK")
conf.get_default().auth_token = os.environ["NGROK"]
public_url = ngrok.connect(8000).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

# = START OLLAMA =

!ollama pull llama3.2